### Задание выполнила Колесниченко Ксения Сергеевна.
Контакты https://www.linkedin.com/in/kseniia-kolesnichenko/



За основу взято решение, предоставленное на Habr https://habr.com/ru/post/414613/
Несмотря на всю важность предобработки данных и создания новых переменных, в качестве  источника данных загружены уже предобработанные в указанном кернеле данные. 

Исходя из анализа имеющегося кернела, наилучшую корреляцию с целевой переменной имеют признаки-факторы EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3. Однако, исходя из анализа кернела, наилучший целевой показатель (площадь под кривой ROC-AUC) получают модели, в которые включены все данные из всех таблиц. 

Известно, что на данный момент наилучший показатель = 0,85, а представленные автором модели дают результат порядка 0,77-0,78

В настоящей работе представлены результаты обучения таких продвинутых алгоритмов, как XGBoost, LGBM, CatBoost, ансамбль стекинга, а так же сложная методика с разделением на фолды, кросс-валидацией и выбором лучшей итерации. 

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV 

import pandas as pd 
from lightgbm import LGBMClassifier

# importing all system modules
import os
import sys
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')


# importing data science modules
import pandas as pd
import numpy as np
import sklearn
import scipy as sp
import pickleshare

# importing graphics modules
import matplotlib.pyplot as plt
import seaborn as sns
# importing personal data science modules
import rand_eda as eda

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import gc

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression


In [2]:
%%time
app_train = pd.read_csv("data_control.csv")
y= pd.read_csv("y_control.csv")
app_test = pd.read_csv("test_control.csv")
app_train.y=y['TARGET']

Wall time: 16.1 s


In [3]:
X=app_train.iloc[:,:-1]
y=app_train.y

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.65, test_size=0.35, random_state=42)

In [ ]:
##app_train['TARGET']=y

In [65]:
%%time
xgb_clf = xgb.XGBClassifier(learning_rate=0.1)
xgb_clf = xgb_clf.fit(X_train, y_train)

probabilities =xgb_clf.predict_proba(X_valid)
submission_xgb = pd.DataFrame({
#    'SK_ID_CURR': app_test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
})
submission_xgb.to_csv("XGBClassifier_full_data_17.csv", index=False)


submission_xgb = pd.DataFrame({
#    'SK_ID_CURR': app_test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
})
y_valid_pred_xgb=submission_xgb
submission_xgb.to_csv("xgbClassifier_17.csv", index=False)

[04:46:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 2min 3s
Parser   : 208 ms


roc_auc_score(y_valid, y_valid_pred_xgb)

In [91]:

probabilities =xgb_clf.predict_proba(app_test.iloc[:,:-1])
submission_xgb065 = pd.DataFrame({
    'SK_ID_CURR': app_test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
})
submission_xgb065.to_csv("XGBClassifier_full_data_17065.csv", index=False)

y_valid_pred_xgb=submission_xgb065

Результат проверки на тестовых данных (на платформе Kaggle) = Score: 0.75608, что говорит
о недообученности модели, по сравнению со значением 0.78 на трейне  

Очень хотелось бы найти наилучшие показатели, но, к сожалению, мощностей не хватает. 

In [49]:
%%time
estimator = XGBClassifier(
    #objective= 'binary:logloss',
    nthread=4,
    seed=42
)
params = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(100, 220, 40),
    'learning_rate': [0.1, 0.01]
}

random_search = RandomizedSearchCV(estimator=estimator, 
                                   param_distributions=params, 
                                   #n_iter=param_comb, 
                                   scoring='roc_auc',
                                   n_jobs=-1, 
                                   cv=5,
                                  verbose=True)


#random_search = RandomizedSearchCV(
#    estimator=estimator,
#    param=param,
#    scoring = 'roc_auc',
#   # n_jobs = 10,
#    cv = 5,
#    verbose=True
#)
random_search.fit(X_train, y_train)
random_search.best_estimator_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


MemoryError: Unable to allocate 221. MiB for an array with shape (181, 159905) and data type int64

In [17]:
from catboost import CatBoostClassifier


In [43]:
%%time
# Initialize CatBoostRegressor
model = CatBoostClassifier(iterations=15,#15
                          learning_rate=0.1,
                          depth=8)#8
# Fit model

model = CatBoostClassifier(iterations=1000) #100

model.fit( X_train, y_train,verbose=False )

# Get predictions
probabilities_CB = model.predict_proba(X_valid)

submission_CB = pd.DataFrame({
#    'SK_ID_CURR': app_test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities_CB]})
y_valid_pred_CB=submission_CB
submission_CB.to_csv("CBClassifier_17.csv", index=False)

Wall time: 1min 31s


In [50]:
 roc_auc_score(y_valid, y_valid_pred_LGBM)

0.7765142491472142

In [92]:

probabilities_CB065 = model.predict_proba(app_test.iloc[:,:-1])
submission_CB065 = pd.DataFrame({
    'SK_ID_CURR': app_test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities_CB065]
})
submission_CB065.to_csv("CB065Classifier_full_data_17065.csv", index=False)

y_valid_pred_CB065=submission_CB065

Score: 0.75608, что тоже говорит о недообученности модели

In [5]:
%%time
import re
data = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = X_valid.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#y = y.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y=y_train

folds = KFold(n_splits=5, shuffle=True, random_state=546789)
oof_preds = np.zeros(data.shape[0])
sub_preds = np.zeros(test.shape[0])

feature_importance_df = pd.DataFrame()

feats = [f for f in data.columns if f not in ['SK_ID_CURR']]

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(data)):
    trn_x, trn_y = data[feats].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = data[feats].iloc[val_idx], y.iloc[val_idx]
    
    clf = LGBMClassifier(
        n_estimators=10000,
        learning_rate=0.03,
        num_leaves=34,
        colsample_bytree=0.9,
        subsample=0.8,
        max_depth=8,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=375,
        silent=-1,
        verbose=-1,
        )
    
    clf.fit(trn_x, trn_y, 
            eval_set= [(trn_x, trn_y), (val_x, val_y)], 
            eval_metric='auc', verbose=100, early_stopping_rounds=100  #30
           )
    
    oof_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
    sub_preds += clf.predict_proba(test[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importances_
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
    del clf, trn_x, trn_y, val_x, val_y
    gc.collect()

print('Full AUC score %.6f' % roc_auc_score(y, oof_preds)) 

y_pred_KFold = sub_preds


test['TARGET'] = sub_preds

y_valid_pred_KFold=test[['SK_ID_CURR', 'TARGET']]

test[['SK_ID_CURR', 'TARGET']].to_csv('y_valid_pred_KFold.csv', index=False)


Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.775271	training's binary_logloss: 0.244215	valid_1's auc: 0.759066	valid_1's binary_logloss: 0.251465
[200]	training's auc: 0.793482	training's binary_logloss: 0.236597	valid_1's auc: 0.768702	valid_1's binary_logloss: 0.24696
[300]	training's auc: 0.802934	training's binary_logloss: 0.23258	valid_1's auc: 0.773035	valid_1's binary_logloss: 0.245183
[400]	training's auc: 0.810329	training's binary_logloss: 0.229645	valid_1's auc: 0.775285	valid_1's binary_logloss: 0.244296
[500]	training's auc: 0.816354	training's binary_logloss: 0.227263	valid_1's auc: 0.776586	valid_1's binary_logloss: 0.24385
[600]	training's auc: 0.821517	training's binary_logloss: 0.22518	valid_1's auc: 0.777608	valid_1's binary_logloss: 0.243485
[700]	training's auc: 0.826285	training's binary_logloss: 0.223316	valid_1's auc: 0.778181	valid_1's binary_logloss: 0.243295
[800]	training's auc: 0.830846	training's binary_logloss: 0.

In [8]:
 roc_auc_score(y_valid, y_valid_pred_KFold.TARGET)

0.7840589710484263

Результат обучения этой модели на тестовых данных уже известен, посчитан автором кернела на Хабре, составляет 0.78230


In [10]:
from lightgbm import LGBMClassifier
import re
test=X_valid
data =  X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y=y_train

clf2 = LGBMClassifier(learning_rate=0.1)
clf2.fit(data, y)

predictions = clf2.predict_proba(test)[:, 1]

# Датафрейм для загрузки
submission = test[['SK_ID_CURR']]
submission['TARGET'] = predictions


y_valid_pred_LGBM=submission['TARGET']

# Сохранение датафрейма
submission.to_csv('y_valid_pred_LGBM.csv', index = False)

In [13]:
 roc_auc_score(y_valid, y_valid_pred_LGBM)

0.7765142491472142

In [93]:
probabilities_LGBM065 = clf2.predict_proba(app_test.iloc[:,:-1])
submission_LGBM065 = pd.DataFrame({
    'SK_ID_CURR': app_test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities_LGBM065]
})
submission_LGBM065.to_csv("LGBM065Classifier_full_data_17065.csv", index=False)

y_valid_pred_LGBM065=submission_LGBM065

In [96]:
submission_LGBM065.head()

,SK_ID_CURR,TARGET
0,100001,0.017118
1,100005,0.068996
2,100013,0.018878
3,100028,0.026924
4,100038,0.181146


В данном случае score 0.75608

Эксперименты со сложением значений модели с некоторыми весами говорит о том, что, вероятно, они могут несколько улучшить модель.
Значения для KFold рассчитаны, сохранены в файле submission_cross.csv

In [60]:

y_valid_pred_KFold.rename(columns={"TARGET": "y_valid_pred_KFold"}, inplace=True)
y_valid_pred_KFold['y_valid_pred_LGBM']=y_valid_pred_LGBM
#y_valid_pred_KFold['KFold-6_LGBM-4']=y_valid_pred_KFold.y_valid_pred_KFold*0.8+y_valid_pred_KFold.y_valid_pred_LGBM*0.2
y_valid_pred_KFold.head()

,SK_ID_CURR,y_valid_pred_KFold,y_valid_pred_LGBM,KFold-6_LGBM-4
245895,384575,0.256136,0.245156,0.253940
98194,214010,0.033505,0.034405,0.033685
36463,142232,0.105034,0.095200,0.103067
249923,389171,0.022364,0.034436,0.024779
158389,283617,0.051554,0.046193,0.050481


In [95]:
submission_cross=pd.read_csv('submission_cross.csv')
submission_cross.head()

,SK_ID_CURR,TARGET
0,100001,0.033623
1,100005,0.112856
2,100013,0.026896
3,100028,0.032239
4,100038,0.134143


In [16]:
 roc_auc_score(y_valid, y_valid_pred_KFold['KFold-6_LGBM-4'])

0.7841834210083324

В связи с нехваткой памяти будет рассчитано в дополнительном файле. Результат составил 0,7814, что меньше имеющихся прогнозов на тестовой выборке.

In [105]:
submission_avg=pd.DataFrame()
submission_avg['SK_ID_CURR']=submission_cross['SK_ID_CURR']

a=submission_cross.TARGET*0.2
b=submission_LGBM065*0.4
c=a+b
#submission_avg['TARGET']=с

MemoryError: Unable to allocate 17.7 GiB for an array with shape (48744, 48744) and data type float64

### Стекинг

In [59]:
%%time
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_valid = X_valid.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#y = y.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#y=y_train

estimators = [
     ('XGB',  xgb.XGBClassifier(random_state=42)),
     ('LGBM', LGBMClassifier(random_state=42)),
     ('CB', CatBoostClassifier(iterations=15,learning_rate=0.1, depth=8,random_state=42)),
    
 ]

clf_stack = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression())
clf_stack.fit(X_train, y_train)
#.score(X_test, y_test)




predictions_stack = clf_stack.predict_proba(X_valid)[:, 1]

# Датафрейм для загрузки
submission_stack = test[['SK_ID_CURR']]
submission_stack['TARGET'] = predictions_stack


y_valid_pred_stack=submission_stack['TARGET']

# Сохранение датафрейма
submission.to_csv('y_valid_pred_stack.csv', index = False)

[04:05:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0:	learn: 0.5966461	total: 446ms	remaining: 6.24s
1:	learn: 0.5200371	total: 637ms	remaining: 4.14s
2:	learn: 0.4610770	total: 838ms	remaining: 3.35s
3:	learn: 0.4162862	total: 1.02s	remaining: 2.8s
4:	learn: 0.3816966	total: 1.19s	remaining: 2.38s
5:	learn: 0.3531171	total: 1.36s	remaining: 2.04s
6:	learn: 0.3320278	total: 1.51s	remaining: 1.73s
7:	learn: 0.3173210	total: 1.68s	remaining: 1.47s
8:	learn: 0.3049157	total: 1.83s	remaining: 1.22s
9:	learn: 0.2962235	total: 2.09s	remaining: 1.04s
10:	learn: 0.2875805	total: 2.4s	remaining: 873ms
11:	learn: 0.2809953	total: 2.57s	remaining: 643ms
12:	learn: 0.2755928	total: 2.76s	remaining: 425ms
13:	learn: 0.2712849	total: 2.95s	remai

In [61]:
 roc_auc_score(y_valid, y_valid_pred_stack)

0.7718652209422182

In [63]:
%%time
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_valid = X_valid.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#y = y.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
#y=y_train

estimators = [
     ('XGB',  xgb.XGBClassifier(random_state=42, learning_rate=0.1)),
     ('LGBM', LGBMClassifier(random_state=42, learning_rate=0.1)),
     ('CB', CatBoostClassifier(iterations=15,learning_rate=0.1, depth=8,random_state=42)),
    
 ]

clf_stack = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression())
clf_stack.fit(X_train, y_train)
#.score(X_test, y_test)




predictions_stack = clf_stack.predict_proba(X_valid)[:, 1]

# Датафрейм для загрузки
submission_stack = test[['SK_ID_CURR']]
submission_stack['TARGET'] = predictions_stack


y_valid_pred_stack=submission_stack['TARGET']

# Сохранение датафрейма
submission.to_csv('y_valid_pred_stack_lr01.csv', index = False)

[04:25:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0:	learn: 0.5966461	total: 459ms	remaining: 6.42s
1:	learn: 0.5200371	total: 726ms	remaining: 4.72s
2:	learn: 0.4610770	total: 899ms	remaining: 3.6s
3:	learn: 0.4162862	total: 1.12s	remaining: 3.09s
4:	learn: 0.3816966	total: 1.32s	remaining: 2.64s
5:	learn: 0.3531171	total: 1.48s	remaining: 2.22s
6:	learn: 0.3320278	total: 1.63s	remaining: 1.86s
7:	learn: 0.3173210	total: 1.84s	remaining: 1.61s
8:	learn: 0.3049157	total: 2.03s	remaining: 1.35s
9:	learn: 0.2962235	total: 2.17s	remaining: 1.09s
10:	learn: 0.2875805	total: 2.35s	remaining: 854ms
11:	learn: 0.2809953	total: 2.6s	remaining: 650ms
12:	learn: 0.2755928	total: 2.77s	remaining: 427ms
13:	learn: 0.2712849	total: 2.95s	remai

In [64]:
 roc_auc_score(y_valid, y_valid_pred_stack)

0.7721865770172549

In [109]:
app_test = app_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

predictions_stack_test = clf_stack.predict_proba(app_test.iloc[:,:-1])
predictions_stack_test = pd.DataFrame({
    'SK_ID_CURR': app_test['SK_ID_CURR'],
    'TARGET':     [ row[1] for row in probabilities]
})
predictions_stack_test.to_csv("stack_test.csv", index=False)

y_valid_pred_stack_test=predictions_stack_test

Score: 0.75608


К сожалению, ни один из предложенных методов не позволил улучшить результат, данный в кернеле. 